In [2]:
!nvidia-smi

Sat Apr 20 15:14:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          On  | 00000001:00:00.0 Off |                    0 |
| N/A   69C    P0             294W / 300W |   3626MiB / 81920MiB |     98%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
from tqdm import tqdm
import json
import random
import torch
import numpy as np
from transformers import AutoTokenizer, AutoConfig, DebertaV2ForSequenceClassification

from bidirectional_mistral import MistralBiModel
from transformers import MistralPreTrainedModel
import torch
import numpy as np
from typing import Optional, List
from torch import nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from transformers.modeling_outputs import SequenceClassifierOutputWithPast

In [5]:
config = AutoConfig.from_pretrained('mistral-191M-mlm/checkpoint-106000')

In [6]:
config.problem_type = "single_label_classification"
config.label2id = {'contradiction': 0, 'entailment': 1}

In [7]:
class MistralForSequenceClassification(MistralPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.model = MistralBiModel(config)
        self.score = nn.Linear(config.hidden_size, self.num_labels, bias=False)

        # Initialize weights and apply final processing
        self.post_init()
        
    def forward(
        self,
        input_ids: torch.LongTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        past_key_values: Optional[List[torch.FloatTensor]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ):
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        transformer_outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        pooled_output = transformer_outputs[0][:, 0]
        logits = self.score(pooled_output)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)
        if not return_dict:
            output = (logits,) + transformer_outputs[2:]
            return ((loss,) + output) if loss is not None else output
        
        return SequenceClassifierOutputWithPast(
            loss=loss,
            logits=logits,
            past_key_values=transformer_outputs.past_key_values,
            hidden_states=transformer_outputs.hidden_states,
            attentions=transformer_outputs.attentions,
        )

In [8]:
model = MistralForSequenceClassification.from_pretrained('mistral-191M-mlm/checkpoint-106000', config = config)

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistral-191M-mlm/checkpoint-106000 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
_ = model.cuda()

In [10]:
tokenizer = AutoTokenizer.from_pretrained('mistral-191M-mlm/checkpoint-106000')

In [11]:
trainable_parameters = [param for param in model.parameters() if param.requires_grad]
trainer = torch.optim.AdamW(trainable_parameters, lr = 2e-5)

In [12]:
train_X, train_Y = [], []
with open('shuffled-train.json') as fopen:
    for l in fopen:
        l = json.loads(l)
        train_X.append(l['src'])
        train_Y.append(l['label'])

In [13]:
test_X, test_Y = [], []
with open('shuffled-test.json') as fopen:
    for l in fopen:
        l = json.loads(l)
        test_X.append(l['src'])
        test_Y.append(l['label'])
        
len(test_X)

16037

In [14]:
batch_size = 10
epoch = 100

In [15]:
best_dev_acc = -np.inf
patient = 1
current_patient = 0

for e in range(epoch):
    pbar = tqdm(range(0, len(train_X), batch_size))
    losses = []
    for i in pbar:
        trainer.zero_grad()
        x = train_X[i: i + batch_size]
        y = np.array(train_Y[i: i + batch_size])
        
        padded = tokenizer(x, truncation = True, padding = True, return_tensors = 'pt', max_length = 1024)
        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()
        
        padded.pop('token_type_ids', None)
            
        loss, pred = model(**padded, return_dict = False)
        loss.backward()
        
        grad_norm = torch.nn.utils.clip_grad_norm_(trainable_parameters, 1.0)
        trainer.step()
        losses.append(float(loss))
        pbar.set_postfix(loss = float(loss))
        
        
    dev_predicted = []
    for i in range(0, len(test_X[:10000]), batch_size):
        x = test_X[i: i + batch_size]
        y = np.array(test_Y[i: i + batch_size])
        padded = tokenizer(x, truncation = True, padding = True, return_tensors = 'pt', max_length = 1024)
        padded['labels'] = torch.from_numpy(y)
        for k in padded.keys():
            padded[k] = padded[k].cuda()
            
        padded.pop('token_type_ids', None)
        
        loss, pred = model(**padded, return_dict = False)
        dev_predicted.append((pred.argmax(axis = 1).detach().cpu().numpy() == y).mean())
        
    dev_predicted = np.mean(dev_predicted)
    
    print(f'epoch: {e}, loss: {np.mean(losses)}, dev_predicted: {dev_predicted}')
    
    if dev_predicted >= best_dev_acc:
        best_dev_acc = dev_predicted
        current_patient = 0
        model.save_pretrained('small')
    else:
        current_patient += 1
    
    if current_patient >= patient:
        break

  6%|▌         | 6258/109357 [35:04<11:27:45,  2.50it/s, loss=0.157] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

  9%|▉         | 9689/109357 [54:03<6:37:16,  4.18it/s, loss=0.445]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 23%|██▎       | 25336/109357 [2:07:02<4:55:32,  4.74it/s, loss=0.474]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change

epoch: 0, loss: 0.300882489479725, dev_predicted: 0.8796000000000002


  0%|          | 354/109357 [00:59<4:21:12,  6.96it/s, loss=0.559]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 10%|▉         | 10523/109357 [27:54<4:21:02,  6.31it/s, loss=0.588]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 18%|█▊        | 19158/109357 [50:23<4:55:38,  5.08it/s, loss=0.526]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change 

epoch: 1, loss: 0.178563735923062, dev_predicted: 0.8804000000000001


  0%|          | 258/109357 [00:44<5:10:39,  5.85it/s, loss=0.916]   


KeyboardInterrupt: 

In [17]:
real_Y = []
for i in tqdm(range(0, len(test_X), batch_size)):
    x = test_X[i: i + batch_size]
    y = np.array(test_Y[i: i + batch_size])
    padded = tokenizer(x, padding = 'longest', return_tensors = 'pt')
    padded['labels'] = torch.from_numpy(y)
    for k in padded.keys():
        padded[k] = padded[k].cuda()
    
    padded.pop('token_type_ids', None)
    loss, pred = model(**padded, return_dict = False)
    real_Y.extend(pred.argmax(axis = 1).detach().cpu().numpy().tolist())

100%|██████████| 1604/1604 [00:37<00:00, 42.36it/s]


In [18]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, test_Y,
        digits = 5
    )
)

              precision    recall  f1-score   support

           0    0.84488   0.90914   0.87583      7165
           1    0.92182   0.86519   0.89261      8872

    accuracy                        0.88483     16037
   macro avg    0.88335   0.88717   0.88422     16037
weighted avg    0.88744   0.88483   0.88511     16037



In [16]:
padded

{'input_ids': tensor([[ 3657,    20,    29,  1541,   863,   482,  6558,  1523, 11768,    15,
          7705,  1816,    21,    15,   704,  5359,   436, 15246,  3534, 13870,
         28305,   365, 15559,    17,  3657,    21,    29, 15577, 19512,   418,
          5079,  1267, 13904,  6332,   331, 20385,  8406,  8948,    17,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [ 3657,    20,    29,  8080, 16965,    17,  3657,    21,    29, 23562,
           954,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
    

In [17]:
tokenizer.model_input_names = ['input_ids', 'attention_mask']

In [23]:
tokenizer.push_to_hub('mesolitica/mnli-malaysian-mistral-191M-MLM-512', safe_serialization = True)

CommitInfo(commit_url='https://huggingface.co/mesolitica/mnli-malaysian-mistral-191M-MLM-512/commit/40f80c41e4ab83409e606afb1df5b12c44e680d7', commit_message='Upload tokenizer', commit_description='', oid='40f80c41e4ab83409e606afb1df5b12c44e680d7', pr_url=None, pr_revision=None, pr_num=None)

In [24]:
model.push_to_hub('mesolitica/mnli-malaysian-mistral-191M-MLM-512', safe_serialization = True)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/665M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/mnli-malaysian-mistral-191M-MLM-512/commit/e9f61b127b9174752b2a63665a54ea555d83d5c6', commit_message='Upload MistralForSequenceClassification', commit_description='', oid='e9f61b127b9174752b2a63665a54ea555d83d5c6', pr_url=None, pr_revision=None, pr_num=None)